# SSA H6 Benchmark Comparison Analysis

This notebook compares PolicyEngine Option 1 (Full Repeal) estimates with Social Security Administration (SSA) H6 benchmarks:

**H6**: Phase out and eliminate federal income taxation of OASDI benefits that is credited to the OASI and DI Trust Funds from 2045-2054

**PolicyEngine Option 1 (Full Repeal)** is a reasonable proxy for H6 since both policies eliminate taxation of Social Security benefits, though H6 phases this in over 2045-2054.

## Analysis Coverage

Decadal analysis (every 10 years) from 2030-2100 showing:
- Total Social Security benefits paid
- Taxable payroll (PolicyEngine vs SSA projections)
- Revenue changes (Option 1 vs H6)

In [ ]:
# Import libraries
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_us import Microsimulation
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")
print(f"Project root: {project_root}")

## 1. Load PolicyEngine Revenue Impacts

Load Option 1 revenue impacts for all years.

In [ ]:
# Load revenue impacts data
data_path = project_root / 'data' / 'revenue_impacts.csv'
revenue_df = pd.read_csv(data_path)

print(f"Loaded {len(revenue_df)} rows from {data_path}")
print(f"Years available: {revenue_df['year'].min()}-{revenue_df['year'].max()}")

## 2. SSA Benchmark Data

Load SSA projected taxable payroll and H6 estimates.

In [ ]:
# Decadal years for analysis
decadal_years = [2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]

# SSA projected taxable payroll from 2024 Trustees Report Table VI.G6
# (Intermediate assumptions, in billions)
ssa_projected_payroll = {
    2030: 17_803,
    2040: 27_337,
    2050: 34_419,
    2060: 41_836,
    2070: 50_887,
    2080: 79_177,
    2090: 126_652,
    2100: 191_588,
}

# SSA H6 estimates (change in annual balance as % of taxable payroll)
# Source: https://www.ssa.gov/oact/solvency/provisions/tables/table_run133.html
ssa_h6_pct_payroll = {
    2030: None,  # TODO: Add from SSA table
    2040: None,  # TODO: Add from SSA table
    2050: None,  # TODO: Add from SSA table
    2060: -1.04,
    2070: None,  # TODO: Add from SSA table
    2080: None,  # TODO: Add from SSA table
    2090: None,  # TODO: Add from SSA table
    2100: -1.12,
}

print("SSA benchmark data loaded")
print(f"Years: {decadal_years}")
print(f"SSA projected payroll range: ${ssa_projected_payroll[2030]:,}B (2030) to ${ssa_projected_payroll[2100]:,}B (2100)")

## 3. Calculate Decadal Metrics

For each decadal year, calculate:
- Total Social Security benefits
- Taxable payroll
- Revenue impacts

In [ ]:
# Calculate total SS benefits and taxable payroll for each decadal year
print("="*80)
print("CALCULATING DECADAL METRICS (Option 1 vs H6)")
print("="*80)

decadal_metrics = []

for year in decadal_years:
    print(f"\nProcessing {year}...")
    
    # Load dataset for this year
    dataset_path = f'hf://policyengine/test/{year}.h5'
    print(f"  Loading dataset: {dataset_path}")
    sim = Microsimulation(dataset=dataset_path)
    
    # Calculate total Social Security benefits
    print(f"  Computing social_security benefits...")
    ss_benefits = sim.calculate("social_security")
    total_ss_benefits = ss_benefits.sum()
    
    # Calculate taxable payroll
    print(f"  Computing taxable payroll...")
    taxable_ss_earnings = sim.calculate("taxable_earnings_for_social_security")
    taxable_self_employment = sim.calculate("social_security_taxable_self_employment_income")
    total_taxable_payroll = taxable_ss_earnings.sum() + taxable_self_employment.sum()
    
    # Get revenue impact from loaded data
    pe_row = revenue_df[
        (revenue_df['reform_name'] == 'option1') &
        (revenue_df['year'] == year) &
        (revenue_df['scoring_type'] == 'dynamic')
    ]
    
    if len(pe_row) > 0:
        revenue_impact = pe_row.iloc[0]['revenue_impact']
        baseline_revenue = pe_row.iloc[0]['baseline_revenue']
        reform_revenue = pe_row.iloc[0]['reform_revenue']
    else:
        revenue_impact = None
        baseline_revenue = None
        reform_revenue = None
    
    decadal_metrics.append({
        'year': year,
        'total_ss_benefits_billions': total_ss_benefits / 1e9,
        'pe_taxable_payroll_billions': total_taxable_payroll / 1e9,
        'ssa_taxable_payroll_billions': ssa_projected_payroll[year],
        'pe_baseline_revenue_billions': baseline_revenue,
        'pe_reform_revenue_billions': reform_revenue,
        'pe_revenue_impact_billions': revenue_impact,
    })
    
    print(f"  SS Benefits: ${total_ss_benefits / 1e9:,.2f}B")
    print(f"  PE Taxable Payroll: ${total_taxable_payroll / 1e9:,.2f}B")
    print(f"  SSA Taxable Payroll: ${ssa_projected_payroll[year]:,}B")
    if revenue_impact:
        print(f"  Revenue Impact: ${revenue_impact:.2f}B")

decadal_df = pd.DataFrame(decadal_metrics)

print("\n" + "="*80)
print("Decadal metrics calculation complete")
print("="*80)

## 4. Compare Option 1 to SSA H6 Benchmark

In [ ]:
# Add SSA H6 comparison
decadal_df['ssa_h6_pct_payroll'] = decadal_df['year'].map(ssa_h6_pct_payroll)
decadal_df['ssa_h6_revenue_billions'] = (
    decadal_df['ssa_h6_pct_payroll'] / 100 * decadal_df['ssa_taxable_payroll_billions']
)

# Calculate PE impact as % of SSA taxable payroll
decadal_df['pe_pct_ssa_payroll'] = (
    decadal_df['pe_revenue_impact_billions'] / decadal_df['ssa_taxable_payroll_billions'] * 100
)

# Calculate difference
decadal_df['diff_pct_payroll'] = (
    decadal_df['pe_pct_ssa_payroll'] - decadal_df['ssa_h6_pct_payroll']
)
decadal_df['diff_revenue_billions'] = (
    decadal_df['pe_revenue_impact_billions'] - decadal_df['ssa_h6_revenue_billions']
)

# Display comprehensive comparison
print("\n" + "="*80)
print("DECADAL COMPARISON: Option 1 (PE) vs H6 (SSA)")
print("="*80)
print("\nAll revenue figures in billions ($B)")
print("Percentages shown as % of SSA projected taxable payroll\n")

display_df = decadal_df[[
    'year',
    'total_ss_benefits_billions',
    'pe_taxable_payroll_billions',
    'ssa_taxable_payroll_billions',
    'pe_revenue_impact_billions',
    'ssa_h6_revenue_billions',
    'diff_revenue_billions',
    'pe_pct_ssa_payroll',
    'ssa_h6_pct_payroll',
    'diff_pct_payroll'
]].copy()

# Format for display
display_df.columns = [
    'Year',
    'Total SS Benefits ($B)',
    'PE Taxable Payroll ($B)',
    'SSA Taxable Payroll ($B)',
    'PE Revenue Impact ($B)',
    'SSA H6 Impact ($B)',
    'Difference ($B)',
    'PE (% SSA Payroll)',
    'SSA H6 (% Payroll)',
    'Diff (pp)'
]

print(display_df.to_string(index=False))

## 5. Visualizations

In [ ]:
# Create decadal comparison visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Total Social Security Benefits by Year',
        'Revenue Impact: Option 1 (PE) vs H6 (SSA)',
        'Taxable Payroll: PolicyEngine vs SSA Projections',
        'Revenue Impact as % of SSA Payroll'
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

years = decadal_df['year']

# 1. Total SS Benefits
fig.add_trace(
    go.Scatter(
        x=years,
        y=decadal_df['total_ss_benefits_billions'],
        mode='lines+markers',
        name='SS Benefits',
        line=dict(color='#06BEE1', width=3),
        marker=dict(size=8),
        text=decadal_df['total_ss_benefits_billions'].apply(lambda x: f"${x:,.0f}B"),
        hovertemplate='<b>Year:</b> %{x}<br><b>Benefits:</b> %{text}<extra></extra>'
    ),
    row=1, col=1
)

# 2. Revenue Impact comparison (only where SSA data exists)
h6_data = decadal_df[decadal_df['ssa_h6_revenue_billions'].notna()]

fig.add_trace(
    go.Scatter(
        x=years,
        y=decadal_df['pe_revenue_impact_billions'],
        mode='lines+markers',
        name='PolicyEngine Option 1',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=True
    ),
    row=1, col=2
)

if len(h6_data) > 0:
    fig.add_trace(
        go.Scatter(
            x=h6_data['year'],
            y=h6_data['ssa_h6_revenue_billions'],
            mode='markers',
            name='SSA H6 Benchmark',
            marker=dict(color='#A23B72', size=12, symbol='diamond'),
            showlegend=True
        ),
        row=1, col=2
    )

# 3. Taxable Payroll comparison
fig.add_trace(
    go.Scatter(
        x=years,
        y=decadal_df['pe_taxable_payroll_billions'],
        mode='lines+markers',
        name='PolicyEngine',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=False
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=years,
        y=decadal_df['ssa_taxable_payroll_billions'],
        mode='lines+markers',
        name='SSA Projection',
        line=dict(color='#F18F01', width=3, dash='dash'),
        marker=dict(size=8),
        showlegend=True
    ),
    row=2, col=1
)

# 4. Revenue Impact as % of payroll
fig.add_trace(
    go.Scatter(
        x=years,
        y=decadal_df['pe_pct_ssa_payroll'],
        mode='lines+markers',
        name='PolicyEngine',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8),
        showlegend=False,
        text=decadal_df['pe_pct_ssa_payroll'].apply(lambda x: f"{x:.2f}%"),
        hovertemplate='<b>Year:</b> %{x}<br><b>PE Impact:</b> %{text}<extra></extra>'
    ),
    row=2, col=2
)

if len(h6_data) > 0:
    fig.add_trace(
        go.Scatter(
            x=h6_data['year'],
            y=h6_data['ssa_h6_pct_payroll'],
            mode='markers',
            name='SSA H6',
            marker=dict(color='#A23B72', size=12, symbol='diamond'),
            showlegend=False,
            text=h6_data['ssa_h6_pct_payroll'].apply(lambda x: f"{x:.2f}%"),
            hovertemplate='<b>Year:</b> %{x}<br><b>SSA H6:</b> %{text}<extra></extra>'
        ),
        row=2, col=2
    )

# Update axes
fig.update_yaxes(title_text="Billions ($)", row=1, col=1)
fig.update_yaxes(title_text="Revenue Impact ($B)", row=1, col=2)
fig.update_yaxes(title_text="Taxable Payroll ($B)", row=2, col=1)
fig.update_yaxes(title_text="% of SSA Payroll", row=2, col=2)

for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Year", row=i, col=j)

# Update layout
fig.update_layout(
    height=900,
    title_text="Decadal Analysis: Option 1 vs H6 (2030-2100)<br><sub>Every 10 years comparison</sub>",
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

## 6. Export Results

In [ ]:
# Create output directory if it doesn't exist
output_dir = project_root / 'data'
output_dir.mkdir(exist_ok=True)

# Export decadal comparison data
decadal_output_file = output_dir / 'option1_h6_decadal_comparison.csv'
decadal_df.to_csv(decadal_output_file, index=False)
print(f"Decadal comparison exported to: {decadal_output_file}")

# Display summary for years with SSA benchmark data
print("\n" + "="*80)
print("KEY FINDINGS (Years with SSA H6 Benchmark Data)")
print("="*80)

ssa_years = decadal_df[decadal_df['ssa_h6_revenue_billions'].notna()]
for _, row in ssa_years.iterrows():
    print(f"\nYear {row['year']}:")
    print(f"  Total SS Benefits: ${row['total_ss_benefits_billions']:,.2f}B")
    print(f"  PE Taxable Payroll: ${row['pe_taxable_payroll_billions']:,.2f}B")
    print(f"  SSA Taxable Payroll: ${row['ssa_taxable_payroll_billions']:,}B")
    print(f"  PolicyEngine Option 1: ${row['pe_revenue_impact_billions']:.2f}B ({row['pe_pct_ssa_payroll']:.3f}% of SSA payroll)")
    print(f"  SSA H6 Benchmark: ${row['ssa_h6_revenue_billions']:.2f}B ({row['ssa_h6_pct_payroll']:.3f}% of SSA payroll)")
    print(f"  Difference: ${row['diff_revenue_billions']:+.2f}B ({row['diff_pct_payroll']:+.3f} percentage points)")

print("\nAnalysis complete!")